# TASK

HuggingFace로 MNLI 모순 분류 구현하기

Task: Multi-genre Natural Language Inference (MNLI)
문제 설명:
두 개의 문장(전제(premise)와 가설(hypothesis))이 주어졌을 때, 두 문장이 서로 논리적으로 연결되어 있는지, 모순되는지, 혹은 관련이 없는지를 판별하는 문제입니다.

- **입력**: premise에 해당하는 문장과 hypothesis에 해당하는 문장 두 개가 입력으로 들어옵니다.
- **출력:** 분류 문제로, 두 문장이 들어왔을 때 다음 세 가지를 예측하시면 됩니다.
    - **Entailment:** 두 문장에 논리적 모순이 없습니다.
    - **Neutral:** 두 문장은 논리적으로 관련이 없습니다.
    - **Contradiction:** 두 문장 사이에 논리적 모순이 존재합니다.


In [10]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer

## Dataset 준비

논리적 모순 분류 dataset을 준비하겠습니다.
Dataset은 `nyu-mll/glue`을 활용합니다.

In [11]:
glues = load_dataset("nyu-mll/glue", "mnli")
glues["train"][1]
# print(data['train']['features']['label'])
print(glues)
print(glues['train'].features['label'])

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})
ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)


어떻게 합치지

두 문장을 `premise` , `hypothesis`에 저장하고 있음
label은 names=['entailment', 'neutral', 'contradiction'] 를 저장함


# 모델 선정

테스크에 맞는 태그를 찾아서 가장 많이 사용하는 모델을 가져옴


BAAI/bge-m3 는 허깅페이스 지원안해서 안씀

http://github.com/FlagOpen/FlagEmbedding

In [12]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"


tokenizer = AutoTokenizer.from_pretrained(model_name)

# 해결 방식 설계

문장의 의미를 해석하고 관련 있음 , 관련 없음, 관련 있고 모순됨 을 구분해야한다  
토크나이저의 경우 언어 모델의 특성 상 모델에 종속적이다
그래서 AutoTokenizer.from_pretrained 로 토크나이저를 가져오는 것이라고 해석했다


https://www.sbert.net/docs/cross_encoder/training_overview.html

In [13]:
max_length = 256

# 2. 전처리 함수 정의 (수정된 버전)
def preprocess_function(data):
    # 배치 데이터에서 직접 추출
    premises = data["premise"]
    hypotheses = data["hypothesis"]
    
    model_inputs = tokenizer(
        premises,
        hypotheses,
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    
    model_inputs["labels"] = data["label"]
    return model_inputs

# 3. 전체 데이터셋에 전처리 적용
tokenized_dataset = glues.map(
    preprocess_function, 
    batched=True,
    remove_columns=glues["train"].column_names  # 원본 컬럼 제거
)

# 4. 필요한 데이터셋 분할
train_dataset = tokenized_dataset["train"]
val_matched_dataset = tokenized_dataset["validation_matched"]
val_mismatched_dataset = tokenized_dataset["validation_mismatched"]

# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')



Map: 100%|██████████| 9847/9847 [00:00<00:00, 13191.17 examples/s]


In [14]:
train_dataset[0]

{'input_ids': [101,
  17158,
  2135,
  6949,
  8301,
  25057,
  2038,
  2048,
  3937,
  9646,
  1011,
  4031,
  1998,
  10505,
  1012,
  102,
  4031,
  1998,
  10505,
  2024,
  2054,
  2191,
  6949,
  8301,
  25057,
  2147,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

기계번역과 같이 입력과 출력을 모두 tokenize하는 경우에는 출력 text를 `tokenizer`의 `text_target`인자로 넘겨주면 됩니다. 이처럼 기계 번역 dataset을 불러오는 것은 imdb dataset과 별반 다르지 않습니다.

마지막으로 `data_collator`를 다음과 같이 구현합니다.

In [15]:
# 1. 데이터셋 기본 정보 확인
print("=== 데이터셋 구조 확인 ===")
print(tokenized_dataset)
print("\n=== 학습 데이터 크기 ===")
print(f"학습 데이터 수: {len(tokenized_dataset['train'])}")

# 2. 실제 데이터 샘플 확인
print("\n=== 데이터 샘플 확인 ===")
sample_idx = 0  # 첫 번째 샘플 확인
sample = tokenized_dataset['train'][sample_idx]

# 3. 토큰화된 텍스트 복원
print("\n=== 토큰화된 텍스트 복원 ===")
decoded_premise = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
decoded_hypothesis = tokenizer.decode(sample['labels'], skip_special_tokens=True)
print(f"Premise: {decoded_premise}")
print(f"Hypothesis: {decoded_hypothesis}")

# 4. 입력 형태 확인
print("\n=== 입력 데이터 형태 확인 ===")
print(f"input_ids 길이: {len(sample['input_ids'])}")
print(f"attention_mask 길이: {len(sample['attention_mask'])}")
print(f"레이블: {sample['labels']}")

# 5. 특수 토큰 확인
print("\n=== 특수 토큰 확인 ===")
special_tokens = tokenizer.special_tokens_map
print(f"특수 토큰: {special_tokens}")

# 6. 패딩 확인
print("\n=== 패딩 확인 ===")
padding_count = sample['attention_mask'].count(0)
print(f"패딩 토큰 수: {padding_count}")
print(f"실제 토큰 수: {len(sample['input_ids']) - padding_count}")

# 7. 레이블 분포 확인
print("\n=== 레이블 분포 확인 ===")
from collections import Counter
labels = [example['labels'] for example in tokenized_dataset['train']]
label_distribution = Counter(labels)
print(f"레이블 분포: {dict(label_distribution)}")

=== 데이터셋 구조 확인 ===
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9847
    })
})

=== 학습 데이터 크기 ===
학습 데이터 수: 392702

=== 데이터 샘플 확인 ===

=== 토큰화된 텍스트 복원 ===
Premise: conceptually cream skimming has two basic dimensions - product and geography. product and geography are what make cream skimming work.
Hypothesis: [unused0]

=== 입력 데이터 형태 확인 ===
in

In [16]:
from transformers import DataCollatorForSeq2Seq
from transformers import DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


## Model 구현

이번에는 기계번역을 위한 sequence-to-sequence 모델을 구현하겠습니다.

In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # MNLI는 3개 라벨


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sequence-to-sequence 모델은 위와 같이 `AutoModelForSeq2SeqLM`을 활용하면 됩니다.
Pre-trained 모델로 `google-t5/t5-small`이라는 것을 사용하고 있습니다.
이 코드 하나로 우리는 기계 번역과 같은 문제를 풀 수 있는 encoder-decoder 구조의 모델을 구현할 수 있습니다.

## 학습

마지막으로 학습 코드를 구현하겠습니다.
학습은 감정 분석과 똑같이 `training_args`를 정의하고 `Trainer`로 이전에 구현한 것들을 모두 수합하면 됩니다.

In [23]:
# 8. 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./results/minilm-mnli",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    # 간단한 설정
    save_steps=500,
    logging_steps=100,
    # load_best_model_at_end 옵션 제거
    metric_for_best_model="accuracy",
    push_to_hub=False,
    resume_from_checkpoint=True
)


/var/folders/t4/5cn643kn7ls_6vyl6y51gnnm0000gn/T/ipykernel_83833/238460600.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    # 기본 정확도
    accuracy = accuracy_score(labels, predictions)
    
    # 클래스별 정밀도, 재현율, F1 점수
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None, 
                                                            labels=[0, 1, 2])
    
    # 가중 평균 메트릭
    weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted')
    
    # 혼동 행렬 계산 (로깅용이 아닌 콘솔 출력용)
    conf_mat = confusion_matrix(labels, predictions)
    print("Confusion Matrix:")
    print(conf_mat)
    
    # 클래스 이름
    label_names = ['entailment', 'neutral', 'contradiction']
    
    # 결과 딕셔너리 (숫자형 값만 포함)
    results = {
        'accuracy': accuracy,
        'weighted_f1': weighted_f1,
        'weighted_precision': weighted_precision, 
        'weighted_recall': weighted_recall
    }
    
    # 클래스별 메트릭 추가
    for i, label in enumerate(label_names):
        results[f'{label}_precision'] = precision[i]
        results[f'{label}_recall'] = recall[i]
        results[f'{label}_f1'] = f1[i]
    
    return results

In [ ]:

# 10. Trainer 초기화 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_matched_dataset,  # 검증 데이터셋으로 matched 사용
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

마지막으로 학습하면 다음과 같습니다.

In [24]:
# 11. 모델 학습
trainer.train()

# 12. 모델 평가 - validation_mismatched 데이터셋도 평가
mismatched_results = trainer.evaluate(val_mismatched_dataset)
print(f"Mismatched validation results: {mismatched_results}")
trainer.save_model()

Step,Training Loss
100,1.095000
200,1.050500
300,1.004700
400,0.980000
500,0.940500
600,0.906000
700,0.883800
800,0.861700
900,0.842700
1000,0.839700


KeyboardInterrupt: 

학습 결과를 보면 다음과 같습니다.

보시다시피 성능이 그렇게 좋지 않습니다. 아마 더 많은 학습 시간을 요구하는 듯 합니다.

하지만 중요한 것은 data부터 모델, 학습 코드 구현까지 매우 쉽게 했다는 것입니다.

In [ ]:

import matplotlib.pyplot as plt
# 클래스별 F1 점수 시각화
def plot_class_metrics(eval_results):
    class_names = ['entailment', 'neutral', 'contradiction']
    f1_scores = [eval_results[f'{label}_f1'] for label in class_names]
    precision_scores = [eval_results[f'{label}_precision'] for label in class_names]
    recall_scores = [eval_results[f'{label}_recall'] for label in class_names]
    
    # 그래프 생성
    plt.figure(figsize=(12, 6))
    x = np.arange(len(class_names))
    width = 0.25
    
    plt.bar(x - width, precision_scores, width, label='Precision')
    plt.bar(x, recall_scores, width, label='Recall')
    plt.bar(x + width, f1_scores, width, label='F1')
    
    plt.xlabel('클래스')
    plt.ylabel('점수')
    plt.title('클래스별 성능 메트릭')
    plt.xticks(x, class_names)
    plt.legend()
    plt.tight_layout()
    plt.savefig('class_metrics.png')
    plt.show()


In [ ]:
# 종합 결과 표 출력
print("\n===== 종합 평가 결과 =====")
print(f"{'데이터셋':<20} {'정확도':<10} {'가중F1':<10} {'가중정밀도':<10} {'가중재현율':<10}")
print("-" * 70)

datasets = {
    "Validation Matched": val_matched_dataset, 
    "Validation Mismatched": val_mismatched_dataset,
    "Test Matched": tokenized_dataset["test_matched"],
    "Test Mismatched": tokenized_dataset["test_mismatched"]
}

for name, dataset in datasets.items():
    results = trainer.evaluate(eval_dataset=dataset)
    print(f"{name:<20} {results['eval_accuracy']:<10.4f} {results['eval_weighted_f1']:<10.4f} "
          f"{results['eval_weighted_precision']:<10.4f} {results['eval_weighted_recall']:<10.4f}")